## Word2Vec 학습

Gensim의 Word2Vec을 학습하기 위해서는 list of str 형식의 input이 필요합니다. 하지만 모든 리뷰들을 메모리에 올리지 않고도 학습할 수 있습니다. generator인 with open을 이용하여 __iter__를 오버라이딩한 CommentWord2Vec이라는 클래스를 생성합니다. 

__iter__에서 

    for doc in f:
        movie_idx, text, score = doc.split('\t')
        yield text.split()

을 수행하면, 파일 f로부터 한 줄을 읽어 tap으로 텍스트를 구분한 뒤, 그 텍스트를 띄어쓰기 기준으로 잘라낸 token list가 for loop 안에서 출력됩니다. 

__iter__는 아래 구문이 작동할 수 있도록 해주는 파이썬 내장함수 입니다. 

    for doc in word2vec_corpus:
        print(doc)

토크나이징이 된 리뷰들을 CommentWord2Vec의 fname으로 입력하였기 때문에, 띄어쓰기 만으로 단어열을 yield 할 수 있습니다. 이러한 작업은 반복이 되기 때문에 raw corpus와 tokenized corpus를 구분하여 저장하는 것이 좋습니다. 

In [4]:
import os
    
class CommentWord2Vec:
    
    def __init__(self, fname):
        self.fname = fname
        if not os.path.isfile(fname):
            print('File not found: %s' % fname)
        
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for doc in f:
                movie_idx, text, score = doc.split('\t')
                yield text.split()
                
tokenized_corpus_fname = 'data/merged_comments_tokenized.txt'                
word2vec_corpus = CommentWord2Vec(tokenized_corpus_fname)

for num_doc, doc in enumerate(word2vec_corpus):
    print(doc)
    if num_doc > 5: break

['명불허전']
['왠지', '고사', '피의', '중간', '고사', '보다', '재미', '가', '없을', '듯해요', '만약', '보게', '된다', '면', '실망', '할듯']
['티아라', '사랑', '해', 'ㅜ']
['황정', '음', '윤시윤', '지붕킥', '인연', '김수', '로', '티아라지연', '공부', '의신', '인연', '너무', '너무', '재미', '있어', '요']
['기대', '완전']
['기대', '지연', '나온다']
['고사', '는', '1편', '으로', '끝', '났어', '야', '했다', '전작', '을', '어설프게', '따라', '할', '거', '였으면']


Gensim의 Word2Vec을 이용합니다. 미리 만들어둔 word2vec_corpus를 Word2Vec의 argument로 입력합니다. default parameters를 이용하여 Word2Vec을 학습힙니다. 

Word2Vec의 arguments 중에서 중요한 것들은 아래와 같습니다. 

- size: 단어의 임베딩 공간의 크기
- alpha: learning rate
- window: 한 단어의 좌/우의 문맥 크기
- min_count: 모델이 학습할 단어의 최소 출현 빈도수
- max_vocab_size: None이 아닌 숫자를 입력하면 빈도수 기준으로 상위 max_vocab_size 개수만큼의 단어만 학습
- sg: 1이면 skipgram 이용
- negative: negative sampling에서 negative sample의 개수

In [5]:
from gensim.models import Word2Vec
import pickle

word2vec_model = Word2Vec(word2vec_corpus)

with open('movie_review_word2vec_model.pkl', 'wb') as f:
    pickle.dump(word2vec_model, f)

In [6]:
from gensim.models import Word2Vec
import pickle

with open('movie_review_word2vec_model.pkl', 'rb') as f:
    word2vec_model = pickle.load(f)

학습된 Word2Vec모델의 .most_similar(단어, topn) 함수는 입력된 단어에 대하여 가장 비슷한 topn개의 다른 단어들과 유사도를 출력합니다. 

아래의 에제에서 '영화'와 가장 비슷한 단어는 '애니'이며, 유사도는 0.7270입니다

In [7]:
word2vec_model.most_similar('영화', topn=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('애니', 0.7243291735649109),
 ('애니메이션', 0.6834967732429504),
 ('작품', 0.6627761125564575),
 ('명화', 0.6315555572509766),
 ('드라마', 0.6142902374267578),
 ('수작', 0.5733400583267212),
 ('기회', 0.5714430809020996),
 ('영회', 0.5696191191673279),
 ('에니메이션', 0.5657410621643066),
 ('블록버스터', 0.564671516418457)]

In [8]:
word2vec_model.most_similar('하정우', topn=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('송강호', 0.9149550199508667),
 ('공유', 0.864500880241394),
 ('이정재', 0.858720600605011),
 ('황정민', 0.8533148169517517),
 ('유아인', 0.8330795168876648),
 ('설경구', 0.8123204112052917),
 ('송광호', 0.8105043172836304),
 ('조진웅', 0.809227705001831),
 ('김명민', 0.8082171678543091),
 ('박신양', 0.8042221069335938)]

In [9]:
word2vec_model.most_similar('1점', topn=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('일점', 0.8869317173957825),
 ('별1개', 0.8459450006484985),
 ('별반개', 0.8441183567047119),
 ('별한개', 0.8436195850372314),
 ('2점', 0.843600869178772),
 ('3점', 0.8047592043876648),
 ('4점', 0.7883341312408447),
 ('5점', 0.7829416990280151),
 ('6점', 0.7369924187660217),
 ('별2개', 0.7319265007972717)]

In [10]:
word2vec_model.most_similar('십점', topn=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('백점', 0.823567807674408),
 ('별5개', 0.7963733673095703),
 ('별다섯개', 0.7959965467453003),
 ('구점', 0.7605199813842773),
 ('일점', 0.7397753596305847),
 ('1점', 0.7311681509017944),
 ('만점', 0.7282696962356567),
 ('별1개', 0.7191915512084961),
 ('별반개', 0.7054992914199829),
 ('천점', 0.6894444823265076)]

In [11]:
word2vec_model.most_similar('이동진', topn=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('김현수', 0.8688392639160156),
 ('이용철', 0.8554785847663879),
 ('송경원', 0.8512225151062012),
 ('황진미', 0.8489428162574768),
 ('정시우', 0.8256686925888062),
 ('박평식', 0.7926696538925171),
 ('한겨레', 0.7796696424484253),
 ('허남웅', 0.7785079479217529),
 ('허지웅', 0.7737522125244141),
 ('씨네21', 0.7644486427307129)]

In [12]:
word2vec_model.most_similar('평론가', topn=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('전문가', 0.868761420249939),
 ('기자', 0.858749508857727),
 ('씨네21', 0.83072829246521),
 ('한겨레', 0.7777066230773926),
 ('평론가들', 0.7665365934371948),
 ('평론', 0.7425943613052368),
 ('시네21', 0.7398937344551086),
 ('기자들', 0.7390679121017456),
 ('일베', 0.7267972230911255),
 ('한겨례', 0.703889012336731)]

In [13]:
word2vec_model.most_similar('평론', topn=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('비평', 0.8207073211669922),
 ('마녀사냥', 0.7472244501113892),
 ('평론가', 0.7425943613052368),
 ('비난', 0.7390035390853882),
 ('평가', 0.7378689050674438),
 ('씨네21', 0.7104280591011047),
 ('악평', 0.7081296443939209),
 ('잘난척', 0.7027782201766968),
 ('판단', 0.6838350892066956),
 ('헛소리', 0.6821395754814148)]

In [14]:
word2vec_model.most_similar('스토리', topn=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('줄거리', 0.8774641156196594),
 ('시나리오', 0.8485631346702576),
 ('내용', 0.7518637776374817),
 ('내러티브', 0.7048014402389526),
 ('플롯', 0.6621859669685364),
 ('소재', 0.656524658203125),
 ('CG', 0.6547880172729492),
 ('전개', 0.6423903703689575),
 ('cg', 0.6194503307342529),
 ('영상미', 0.6150219440460205)]

In [15]:
word2vec_model.most_similar('조연', topn=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('조연들', 0.8212900161743164),
 ('단역', 0.718321681022644),
 ('주연', 0.7124711275100708),
 ('조연들의', 0.6839877963066101),
 ('배역들', 0.5847113132476807),
 ('이경영', 0.5693899393081665),
 ('정재영', 0.5636854767799377),
 ('박성웅', 0.5521552562713623),
 ('까메오', 0.5408122539520264),
 ('악역', 0.5397312641143799)]

In [16]:
word2vec_model.most_similar('배우', topn=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('캐릭터', 0.7270505428314209),
 ('케릭터', 0.661278247833252),
 ('톱스타', 0.6452080011367798),
 ('주연', 0.5865276455879211),
 ('주인공', 0.5735197067260742),
 ('연기', 0.5692157745361328),
 ('조연들', 0.5619346499443054),
 ('아이', 0.5592776536941528),
 ('꼬맹이', 0.5537346601486206),
 ('배역', 0.5302594900131226)]

In [17]:
word2vec_model.most_similar('포디', topn=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('4d', 0.8237780928611755),
 ('4D', 0.7838330268859863),
 ('투디', 0.7834957838058472),
 ('2d', 0.7822491526603699),
 ('3디', 0.7751452922821045),
 ('3d', 0.7560898661613464),
 ('4디', 0.7519394755363464),
 ('아이맥스', 0.7426560521125793),
 ('2D', 0.7409471869468689),
 ('imax', 0.7382624745368958)]

In [18]:
word2vec_model.most_similar('영등포', topn=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('용산', 0.9293226003646851),
 ('일산', 0.9032312631607056),
 ('신도림', 0.8980547785758972),
 ('신촌', 0.8943902850151062),
 ('센텀', 0.8889030814170837),
 ('왕십리', 0.8824732303619385),
 ('수원', 0.8818697333335876),
 ('롯데', 0.8782415390014648),
 ('메가박스', 0.8761835098266602),
 ('시네마', 0.8695105314254761)]

In [19]:
word2vec_model.most_similar('메가박스', topn=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('영등포', 0.8761836290359497),
 ('롯데', 0.8687515258789062),
 ('용산', 0.8673617839813232),
 ('일산', 0.8535846471786499),
 ('시네마', 0.8505349159240723),
 ('신촌', 0.8440471887588501),
 ('왕십리', 0.841697096824646),
 ('목동', 0.8348946571350098),
 ('센텀', 0.8170782923698425),
 ('동대문', 0.8157122731208801)]

In [20]:
word2vec_model.most_similar('ocn', topn=10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('OCN', 0.8530149459838867),
 ('설특집', 0.7495620250701904),
 ('특집', 0.7410140037536621),
 ('설날특선', 0.7393440008163452),
 ('아맥', 0.7372578978538513),
 ('공중파', 0.7334888577461243),
 ('케이블', 0.7270175218582153),
 ('tv', 0.7266528606414795),
 ('KBS', 0.7116642594337463),
 ('파일', 0.7082415819168091)]

In [21]:
word2vec_model.most_similar('황정민')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('김명민', 0.8988000154495239),
 ('김윤석', 0.8982965350151062),
 ('유아인', 0.8824373483657837),
 ('조진웅', 0.8750876188278198),
 ('정재영', 0.8706260919570923),
 ('유해진', 0.8681741952896118),
 ('송강호', 0.8603654503822327),
 ('하정우', 0.8533147573471069),
 ('최민식', 0.8532742261886597),
 ('정우성', 0.8522543907165527)]

## Doc2Vec 학습

Doc2Vec을 학습하기 위해서는 각각 문서의 label이 저장되어야 합니다. 이를 위하여 TaggedDocument라는 클래스가 이용됩니다. TaggedDocument는 단어들을 words에, 레이블 정보를 tags에 리스트 형태로 입력합니다

In [ ]:
import os
import pickle
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument


class CommentDoc2Vec:
    
    def __init__(self, fname):
        self.fname = fname
        if not os.path.isfile(fname):
            print('File not found: %s' % fname)
        
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for doc in f:
                movie_idx, text, score = doc.split('\t')
                yield TaggedDocument(words=text.split(), tags=['MOVIE_%s' % movie_idx])

                
doc2vec_corpus = CommentDoc2Vec(tokenized_corpus_fname)
doc2vec_model = Doc2Vec(doc2vec_corpus)

with open('tmp/movie_review_doc2vec_model.pkl', 'wb') as f:
    pickle.dump(doc2vec_model, f)

In [ ]:
import pickle
from gensim.models import Doc2Vec

with open('tmp/movie_review_doc2vec_model.pkl', 'rb') as f:
    doc2vec_model = pickle.load(f)

Doc2Vec은 Word2Vec과 동일하게 단어 벡터를 학습하며, 이를 바탕으로 document vector를 학습합니다. 그렇기 때문에 단어 벡터에 의한 유사도는 동일하게 학습됩니다. 

In [ ]:
doc2vec_model.most_similar('영화', topn=10)

Doc2Vec model의 .docvecs안에는 document vector와 관련된 정보들이 저장되어 있습니다

In [ ]:
len(doc2vec_model.docvecs), len(doc2vec_corpus.movie2docid)

Doctags에 들어있는 offset은 document vector의 임베딩 메트릭스의 row id이며, word_count는 각 태그에 해당하는 문서에 단어가 몇 개 있었는지, doc_count는 각 태그에 해당하는 문서가 몇 번 등장하였는지입니다

In [ ]:
for idx, doctag in sorted(doc2vec_model.docvecs.doctags.items(), key=lambda x:x[1].offset):
    print(idx, doctag)

위에서 MOVIE_59845의 offset이 1임을 확인할 수 있습니다. 그렇기 때문에 아래의 두 구문은 같은 결과가 나옵니다

리스트 안의 tuple 값의 의미는 (doctag, 유사도) 입니다 

In [ ]:
doc2vec_model.docvecs.most_similar('MOVIE_59845')

In [ ]:
doc2vec_model.docvecs.most_similar(1)

Dov2Veco.docvecs.doctags는 document vector의 각 row 기준으로 tag 정보가 저장되어 있습니다. 

In [ ]:
list(doc2vec_model.docvecs.doctags)[:5]

In [ ]:
list(doc2vec_model.docvecs.doctags.items())[:5]

## Doc2Vec 해석하기

In [ ]:
import pickle

with open('tmp/navermovie_info_idx2moviename.pkl', 'rb') as f:
    idx2movie = pickle.load(f)
    
movie2idx = {name:movie_idx for movie_idx, name in idx2movie.items()}

Doc2Vec.docvecs.most_similar에서는 태그로 구분되는 문서들에 대하여 유사한 다른 문서를 찾아줍니다. 

In [ ]:
for similar in doc2vec_model.docvecs.most_similar('MOVIE_134963'):
    print(similar)

In [ ]:
list(doc2vec_model.docvecs.doctags)[:3]

영화 아이디를 영화 제목으로 바꿔서 해석해봅시다

In [ ]:
for doctag in doc2vec_model.docvecs.doctags:
    movie_idx = doctag.split('_')[1]
    name = idx2movie.get(movie_idx, -1)
    print(movie_idx, '\t', name)

영화 리뷰를 기준으로 각 영화를 document vector로 표현하였을 때 라라랜드와 리뷰가 비슷한 영화는 '비긴 어게인', '어바웃 타임' 등입니다

In [ ]:
def id2name(similar):
    idx = similar[0].split('_')[1]
    return (idx2movie.get(idx, 'unknown'), idx, similar[1])


print('라라랜드\n')

for similar in doc2vec_model.docvecs.most_similar('MOVIE_134963'):
    print(id2name(similar))

In [ ]:
print('관상\n')
for similar in doc2vec_model.docvecs.most_similar('MOVIE_93728'):
    print(id2name(similar))

In [ ]:
print('광해 왕이된 남자\n')
for similar in doc2vec_model.docvecs.most_similar('MOVIE_83893'):
    print(id2name(similar))

In [ ]:
print('아바타\n')
for similar in doc2vec_model.docvecs.most_similar('MOVIE_62266'):
    print(id2name(similar))

## t-SNE를 이용한 리뷰가 비슷한 영화의 시각화

In [ ]:
doc2vec_model.docvecs.doctag_syn0.shape, type(doc2vec_model.docvecs.doctag_syn0)

In [ ]:
index2tag = [doc2vec_model.docvecs.index_to_doctag(i) for i in range(172)]
print(index2tag[:5])

tag2index = {tag:index for index, tag in enumerate(index2tag)}

index2name = [idx2movie.get(tag.split('_')[1], -1) for tag in index2tag]
print(index2name[:5])

이전의 문서 시각화 시간에 살펴보았던 TSNE를 이용하여 document vector를 2차원으로 임베딩을 하여 시각화를 수행합니다. 시간이 오래 걸릴 작업은 %%time을 적어 수행 시간을 출력하면 편합니다.

In [ ]:
%%time 

from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
y_tsne = tsne.fit_transform(doc2vec_model.docvecs.doctag_syn0)

y_tsne.shape

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

list(matplotlib.font_manager.get_fontconfig_fonts())[:5]

krfont = {'family' : 'nanumgothic', 'weight' : 'bold', 'size'   : 10}
matplotlib.rc('font', **krfont)

리뷰의 경향이 비슷하다고 확인되었던 '라라랜드', '어바웃타임', '비긴어게인', '뷰티 인사이드' 등은 비슷한 공간에 위치함을 볼 수 있습니다

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(y_tsne[:,0], y_tsne[:,1], color='yellow')

similars = [tag for tag, _ in doc2vec_model.docvecs.most_similar('MOVIE_134963')] + ['MOVIE_134963']

for tag in similars:
    index = tag2index.get(tag, -1)
    if index == -1:
        continue
    name = index2name[index]
    plt.annotate(name, (y_tsne[index, 0], y_tsne[index, 1]))

plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(y_tsne[:,0], y_tsne[:,1], color='yellow')

similars = [tag for tag, _ in doc2vec_model.docvecs.most_similar('MOVIE_93728')] + ['MOVIE_93728']

for tag in similars:
    index = tag2index.get(tag, -1)
    if index == -1:
        continue
    name = index2name[index]
    plt.annotate(name, (y_tsne[index, 0], y_tsne[index, 1]))

plt.show()

액션적인 요소가 많았던 영화들은 앞선 라라랜드나 비긴어게인과 같은 영화들과는 떨어져서, 액션 영화들끼리 뭉쳐있음을 확인할 수 있습니다

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(y_tsne[:,0], y_tsne[:,1], color='yellow')

similars = [tag for tag, _ in doc2vec_model.docvecs.most_similar('MOVIE_62266')] + ['MOVIE_62266']

for tag in similars:
    index = tag2index.get(tag, -1)
    if index == -1:
        continue
    name = index2name[index]
    plt.annotate(name, (y_tsne[index, 0], y_tsne[index, 1]))

plt.show()